In [81]:
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants
import pandas as pd

StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 83, Finished, Available, Finished)

In [82]:
df = spark.read.synapsesql("s_warehouse.dbo.accounts")
df.show()

StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 84, Finished, Available, Finished)

+-----------+-------+------------+----------+
|customer_id|balance|account_type|account_id|
+-----------+-------+------------+----------+
|         45| 1000.5|     Savings|       1.0|
|         12|2500.75|    Checking|       2.0|
|         78| 1500.0|     Savings|       3.0|
|         34|3000.25|    Checking|       4.0|
|         56|  500.0|     Savings|       5.0|
|         23| 1200.5|    Checking|       6.0|
|         89| 800.75|     Savings|       7.0|
|         67| 2200.0|    Checking|       8.0|
|         14| 900.25|     Savings|       9.0|
|         92| 1800.5|    Checking|      10.0|
|          3|1100.75|     Savings|      11.0|
|         81| 2700.0|    Checking|      12.0|
|         29|1300.25|     Savings|      13.0|
|         64| 3200.5|    Checking|      14.0|
|         47| 700.75|     Savings|      15.0|
|         18| 1400.0|    Checking|      16.0|
|         99| 600.25|     Savings|      17.0|
|          5| 1600.5|    Checking|      18.0|
|         76| 400.75|     Savings|

In [ ]:
# Define Delta table creation query
create_table_query = """
CREATE TABLE IF NOT EXISTS Accounts_Gold (
    account_id INT,
    customer_id INT,
    account_type STRING,
    balance FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Accounts_Gold'
"""

# Execute the table creation
spark.sql(create_table_query)

#spark.sql("DROP TABLE IF EXISTS Customers_Gold")




StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 23, Finished, Available, Finished)

DataFrame[]

In [84]:
from pyspark.sql.functions import col, lit, current_timestamp, crc32, concat_ws
from delta.tables import DeltaTable

# Load source data from Warehouse Silver Layer
source_df = spark.read.synapsesql("s_warehouse.dbo.accounts")

# Add metadata and Hash Key
source_df = source_df \
    .withColumn("hash_key", crc32(concat_ws("||", *source_df.columns))) \
    .withColumn("created_date", current_timestamp()) \
    .withColumn("updated_date", current_timestamp()) \
    .withColumn("created_by", lit("fabric")) \
    .withColumn("updated_by", lit("fabric"))


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 86, Finished, Available, Finished)

In [85]:
# Load target Delta Table in Lakehouse (Gold Layer)
target_table_path = "Files/Gold_layer/Accounts_Gold"
target_delta = DeltaTable.forPath(spark, target_table_path)
target_df = target_delta.toDF()


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 87, Finished, Available, Finished)

In [86]:
# Find records that are new or changed by comparing account_id and hash_key
df_diff = source_df.alias("src").join(
    target_df.alias("tgt"),
    (col("src.account_id") == col("tgt.account_id")) & (col("src.hash_key") == col("tgt.hash_key")),
    "anti"
).select("src.*")


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 88, Finished, Available, Finished)

In [87]:
# Apply SCD-1 Merge Logic to Gold Layer Delta Table
target_delta.alias("tgt").merge(
    df_diff.alias("src"),
    "tgt.account_id = src.account_id"
) \
.whenMatchedUpdate(set={
    "account_id": "src.account_id",
    "customer_id": "src.customer_id",
    "account_type": "src.account_type",
    "balance": "src.balance",
    "hash_key": "src.hash_key",
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric-updated")
}) \
.whenNotMatchedInsert(values={
    "account_id": "src.account_id",
    "customer_id": "src.customer_id",
    "account_type": "src.account_type",
    "balance": "src.balance",
    "hash_key": "src.hash_key",
    "created_date": "current_timestamp()",
    "created_by": lit("fabric"),
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric")
}).execute()

df = spark.read.format("delta").load("Files/Gold_layer/Accounts_Gold")

# Convert to Pandas and display as table
pd_df = df.toPandas()
display(pd_df)


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 89, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5067e1d9-0faa-44b2-8e83-c99112b606d9)

In [65]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Customer_Gold (
    customer_id INT,
    first_name STRING,
    last_name STRING,
    address STRING,
    city STRING,
    state STRING,
    zip STRING,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Customer_Gold'
"""
spark.sql(create_table_query)

#spark.sql("DROP TABLE IF EXISTS Customers_Gold")


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 67, Finished, Available, Finished)

DataFrame[]

In [69]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Loan_Gold (
    loan_id INT,
    customer_id INT,
    loan_term STRING,
    loan_amount FLOAT,
    interest_rate FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Loan_Gold'
"""
spark.sql(create_table_query)


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 71, Finished, Available, Finished)

DataFrame[]

In [74]:
create_table_query = """
CREATE TABLE IF NOT EXISTS LoanPayments_Gold (
    payment_id INT,
    loan_id INT,
    payment_date DATE,
    payment_amount FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/LoanPayments_Gold'
"""
spark.sql(create_table_query)


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 76, Finished, Available, Finished)

DataFrame[]

In [77]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Transaction_Gold (
    transaction_id INT,
    account_id INT,
    transaction_date DATE,
    transaction_type STRING,
    transaction_amount FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Transaction_Gold'
"""
spark.sql(create_table_query)



StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 79, Finished, Available, Finished)

DataFrame[]

In [88]:
from pyspark.sql.functions import col, lit, current_timestamp, crc32, concat_ws
from delta.tables import DeltaTable

# Load source data
source_df = spark.read.synapsesql("s_warehouse.dbo.customers")

# Add metadata and Hash Key
source_df = source_df \
    .withColumn("hash_key", crc32(concat_ws("||", *source_df.columns))) \
    .withColumn("created_date", current_timestamp()) \
    .withColumn("updated_date", current_timestamp()) \
    .withColumn("created_by", lit("fabric")) \
    .withColumn("updated_by", lit("fabric"))

# Load target Delta Table
target_table_path = "Files/Gold_layer/Customer_Gold"
target_delta = DeltaTable.forPath(spark, target_table_path)
target_df = target_delta.toDF()

# Find new or updated records
df_diff = source_df.alias("src").join(
    target_df.alias("tgt"),
    (col("src.customer_id") == col("tgt.customer_id")) & (col("src.hash_key") == col("tgt.hash_key")),
    "anti"
).select("src.*")

# Apply SCD-1 Merge Logic
target_delta.alias("tgt").merge(
    df_diff.alias("src"),
    "tgt.customer_id = src.customer_id"
) \
.whenMatchedUpdate(set={
    "customer_id": "src.customer_id",
    "first_name": "src.first_name",
    "last_name": "src.last_name",
    "address": "src.address",
    "city": "src.city",
    "state": "src.state",
    "zip": "src.zip",
    "hash_key": "src.hash_key",
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric-updated")
}) \
.whenNotMatchedInsert(values={
    "customer_id": "src.customer_id",
    "first_name": "src.first_name",
    "last_name": "src.last_name",
    "address": "src.address",
    "city": "src.city",
    "state": "src.state",
    "zip": "src.zip",
    "hash_key": "src.hash_key",
    "created_date": "current_timestamp()",
    "created_by": lit("fabric"),
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric")
}).execute()

df = spark.read.format("delta").load("Files/Gold_layer/Customer_Gold")
pd_df = df.toPandas()
display(pd_df)


StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 90, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 16cfb7b9-ab2c-43cf-b125-9fbd365b960f)

In [89]:
# Load source data
source_df = spark.read.synapsesql("s_warehouse.dbo.loans")

# Add metadata and Hash Key
source_df = source_df \
    .withColumn("hash_key", crc32(concat_ws("||", *source_df.columns))) \
    .withColumn("created_date", current_timestamp()) \
    .withColumn("updated_date", current_timestamp()) \
    .withColumn("created_by", lit("fabric")) \
    .withColumn("updated_by", lit("fabric"))

# Load target Delta Table
target_table_path = "Files/Gold_layer/Loan_Gold"
target_delta = DeltaTable.forPath(spark, target_table_path)
target_df = target_delta.toDF()

# Find new or updated records
df_diff = source_df.alias("src").join(
    target_df.alias("tgt"),
    (col("src.loan_id") == col("tgt.loan_id")) & (col("src.hash_key") == col("tgt.hash_key")),
    "anti"
).select("src.*")

# Apply SCD-1 Merge Logic
target_delta.alias("tgt").merge(
    df_diff.alias("src"),
    "tgt.loan_id = src.loan_id"
) \
.whenMatchedUpdate(set={
    "loan_id": "src.loan_id",
    "customer_id": "src.customer_id",
    "loan_term": "src.loan_term",
    "loan_amount": "src.loan_amount",
    "interest_rate":"src.interest_rate",
    "hash_key": "src.hash_key",
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric-updated")
}) \
.whenNotMatchedInsert(values={
    "loan_id": "src.loan_id",
    "customer_id": "src.customer_id",
    "loan_term": "src.loan_term",
    "loan_amount": "src.loan_amount",
    "interest_rate":"src.interest_rate",
    "hash_key": "src.hash_key",
    "created_date": "current_timestamp()",
    "created_by": lit("fabric"),
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric")
}).execute()

# Verify Data Load
df = spark.read.format("delta").load("Files/Gold_layer/Loan_Gold")
pd_df = df.toPandas()
display(pd_df)



StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 91, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c13ebd37-b061-49c0-ae8b-e64df9d21fa6)

In [90]:
# Load source data
source_df = spark.read.synapsesql("s_warehouse.dbo.loan_payments")

# Add metadata and Hash Key
source_df = source_df \
    .withColumn("hash_key", crc32(concat_ws("||", *source_df.columns))) \
    .withColumn("created_date", current_timestamp()) \
    .withColumn("updated_date", current_timestamp()) \
    .withColumn("created_by", lit("fabric")) \
    .withColumn("updated_by", lit("fabric"))

# Load target Delta Table
target_table_path = "Files/Gold_layer/LoanPayments_Gold"
target_delta = DeltaTable.forPath(spark, target_table_path)
target_df = target_delta.toDF()

# Find new or updated records
df_diff = source_df.alias("src").join(
    target_df.alias("tgt"),
    (col("src.payment_id") == col("tgt.payment_id")) & (col("src.hash_key") == col("tgt.hash_key")),
    "anti"
).select("src.*")

# Apply SCD-1 Merge Logic
target_delta.alias("tgt").merge(
    df_diff.alias("src"),
    "tgt.payment_id = src.payment_id"
) \
.whenMatchedUpdate(set={
    "payment_id": "src.payment_id",
    "loan_id": "src.loan_id",
    "payment_date": "src.payment_date",
    "payment_amount": "src.payment_amount",
    "hash_key": "src.hash_key",
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric-updated")
}) \
.whenNotMatchedInsert(values={
    "payment_id": "src.payment_id",
    "loan_id": "src.loan_id",
    "payment_date": "src.payment_date",
    "payment_amount": "src.payment_amount",
    "hash_key": "src.hash_key",
    "created_date": "current_timestamp()",
    "created_by": lit("fabric"),
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric")
}).execute()

# Verify Data Load
df = spark.read.format("delta").load("Files/Gold_layer/LoanPayments_Gold")
pd_df = df.toPandas()
display(pd_df)



StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 92, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c4dfd22-51a2-40c2-b76f-165cd27d3d91)

In [91]:
# Load source data
source_df = spark.read.synapsesql("s_warehouse.dbo.transactions")

# Add metadata and Hash Key
source_df = source_df \
    .withColumn("hash_key", crc32(concat_ws("||", *source_df.columns))) \
    .withColumn("created_date", current_timestamp()) \
    .withColumn("updated_date", current_timestamp()) \
    .withColumn("created_by", lit("fabric")) \
    .withColumn("updated_by", lit("fabric"))

# Load target Delta Table
target_table_path = "Files/Gold_layer/Transaction_Gold"
target_delta = DeltaTable.forPath(spark, target_table_path)
target_df = target_delta.toDF()

# Find new or updated records
df_diff = source_df.alias("src").join(
    target_df.alias("tgt"),
    (col("src.transaction_id") == col("tgt.transaction_id")) & (col("src.hash_key") == col("tgt.hash_key")),
    "anti"
).select("src.*")

# Apply SCD-1 Merge Logic
target_delta.alias("tgt").merge(
    df_diff.alias("src"),
    "tgt.transaction_id = src.transaction_id"
) \
.whenMatchedUpdate(set={
    "transaction_id": "src.transaction_id",
    "account_id": "src.account_id",
    "transaction_date": "src.transaction_date",
    "transaction_type": "src.transaction_type",
    "transaction_amount": "src.transaction_amount",
    "hash_key": "src.hash_key",
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric-updated")
}) \
.whenNotMatchedInsert(values={
    "transaction_id": "src.transaction_id",
    "account_id": "src.account_id",
    "transaction_date": "src.transaction_date",
    "transaction_type": "src.transaction_type",
    "transaction_amount": "src.transaction_amount",
    "hash_key": "src.hash_key",
    "created_date": "current_timestamp()",
    "created_by": lit("fabric"),
    "updated_date": "current_timestamp()",
    "updated_by": lit("fabric")
}).execute()

# Verify Data Load
df = spark.read.format("delta").load("Files/Gold_layer/Transaction_Gold")
pd_df = df.toPandas()
display(pd_df)



StatementMeta(, 26add48e-d3d5-4217-a5e5-537134d1e4be, 93, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0168da84-b171-4203-9a13-bbdf3ff9b708)